![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 04 - Introducción a la librería Keras

Una vez finalizado la introducción al paradigma de aprendizaje incremental, nos centraremos ahora en el desarrollo y aplicación de modelos basados en redes neuronales profundas, ya que nos permitirán generar modelos que tengan en cuenta el tiempo. Para poder desarrollar  este tipo de redes, emplearemos Tensorflow y el API de Keras3. Dado que la experiencia en este máster estuvo centrada en en `scikit-learn`, emplearemos esta práctica a modo de introducción al uso de Keras con Tensorflow.





La mayor parte del contenido de este Notebook se dedica a explicar el uso básico del API Keras empleando como *backend* Tensorflow. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en su funcionamiento.



**Importante**: El Notebook contiene varios ejercicios sencillos. Deberéis desarrollarlos durante la clase y enviarlos por el aula virtual del curso, en la tarea correspondiente


## Referencias útiles para la práctica

1. [Documentación oficial](https://www.tensorflow.org/learn?hl=es-419) de Tensorflow
2. [Guías](https://keras.io/guides/) de Keras




## 1. Introducción 
**Es habitual que en las plantas industriales nos encontremos con procesos secuenciales altamente no lineales y con grandes dependencias temporales**. Pensad que un subproceso que se está aplicando ahora mismo en una parte de la línea a un producto intermedio, no tendrá efecto en el producto final hasta un tiempo después, ya que es necesario que el producto intermedio avance y  atraviese los diferentes subprocesos productivos hasta llegar al final.

En general los modelos vistos hasta ahora no tienen en cuenta el tiempo. Típicamente son "fotos fijas" de un proceso en el que se recogen las medidas o características de ese momento dado. Por ejemplo, cada observación puede estar compuesta de las medidas de todos los sensores desplegados en la factoría en un segundo concreto.
En un proceso secuencial con dependencias temporales, estaríamos asociando a la variable/s de salida/s, características de entrada que no fueron las que realmente afectaron  y generaron dicha salida.

Típicamente. para poder trabajar en esta situación en una línea de producción tenemos 2 opciones:
1. Podemos **alinear los datos**, es decir, tratar de crear observaciones que cuadren las entradas concretas que afectaron a la salida escogida. Esto es algo tremendamente complicado si no se conoce perfectamente el proceso productivo y sus características.
2. Podemos **trabajar con modelos que recuerden y tengan en cuenta lo que ha pasado en instantes anteriores**.

Trabajar con modelos recurrentes que tengan en cuenta el tiempo es algo que suele beneficiar el desarrollo de predictores en plantas industriales pero son más complicados de desarrollar y entrenar. Librerías típicas como `scikit-learn`no contemplan el  desarrollo de este tipo de modelos pero existen otros frameworks centrados en el desarrollo de redes neuronales profundas en general y que permiten crear redes recurrentes de diferentes tipos. [Tensorflow](https://www.tensorflow.org/?hl=es-419) es uno de los *frameworks* más populares 

In [ ]:

# Importaciones generales
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np


try:
    import seaborn as sns
except ImportError as err:
    !pip install seaborn
    import seaborn as sns



try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

import os

# Asegurarnos de usar Keras 3 con backend TensorFlow
# Es necesario hacerlo antes de cargar Keras
os.environ["KERAS_BACKEND"] = "tensorflow"

# Importaciones de Keras y TensorFlow
try:
    import keras
except ImportError as err:
    !pip install keras
    import keras

try:
    import tensorflow as tf 
except ImportError as err:
    !pip install tensorflow
    import tensorflow as tf 



# Importaciones de Scikit-learn
#Solo para facilitarnos el uso de algunas operaciones típicas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Configuraciones para visualización. No es necesario
plt.style.use('seaborn-v0_8-whitegrid') # Estilo de gráficos
sns.set_palette('viridis') # Paleta de colores


# Verificación de versiones
#Existen diferentes versiones y compatibilidades. Es importantes saber lo que estamos ejecutando
#Para estos ejemplos queremos ejecutar el API de Keras3 (standalone) y tensorflo3>2.16
print(f"Versión de Keras: {keras.__version__}")
print(f"Backend de Keras: {keras.backend.backend()}")
print(f"Versión de TensorFlow: {tf.__version__}")



# Verificar si TensorFlow puede usar GPU en nuestro sistema (opcional, pero bueno saberlo)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
else:
    print("GPU no encontrada, se usará CPU.")

Para el desarrollo del ejemplo emplearemos un *dataset* público de Kaggle: [Concrete Compressive Strength Data Set](https://www.kaggle.com/datasets/elikplim/concrete-compressive-strength-data-set/data).
El repositorio contiene una descripción del mismo y de sus campos:


**Concrete Compressive Strength Data Set**


*Abstract: Concrete is the most important material in civil engineering. The
concrete compressive strength is a highly nonlinear function of age and
ingredients. These ingredients include cement, blast furnace slag, fly ash,
water, superplasticizer, coarse aggregate, and fine aggregate.*

| **Nombre** | **Tipo de dato** | **Medida** | **Descripción** |
|---|---|---|---|
| Cement (component 1) | quantitative | kg in a m³ mixture | Input Variable |
| Blast Furnace Slag (component 2) | quantitative | kg in a m³ mixture | Input Variable |
| Fly Ash (component 3) | quantitative | kg in a m³ mixture | Input Variable |
| Water (component 4) | quantitative | kg in a m³ mixture | Input Variable |
| Superplasticizer (component 5) | quantitative | kg in a m³ mixture | Input Variable |
| Coarse Aggregate (component 6) | quantitative | kg in a m³ mixture | Input Variable |
| Fine Aggregate (component 7) | quantitative | kg in a m³ mixture | Input Variable |
| Age | quantitative | Day (1\~365) | Input Variable |
| **Concrete compressive strength** | quantitative | MPa | Output Variable |



Descargamos la última versión del dataset empleando `kagglehub`.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elikplim/concrete-compressive-strength-data-set")

print("Path to dataset files:", path)
#Nombre del fichero descargado: concrete_data.csv
df_concrete=pd.read_csv(path+"/concrete_data.csv")



Visualizamos los datos cargados. 

**IMPORTANTE**: *el objetivo  de este Notebook es desarrollar modelos con Keras y Tensorflow, por lo que simplificaremos al máximo las fases de análisis y preprocesado. El rendimiento del modelo no es prioritario. Conocéis de materias previas la importancia de analizar y preparar los datos  para mejorar la precisión de los modelos.*

In [ ]:
print(f"Numero de filas: {len(df_concrete)}")
print(f"Numero de características: {len(df_concrete.columns)}")

print("Información general del dataset:")
df_concrete.info()


 # Vistazo rápido a los datos
print("Primeras filas del dataset:")
print(df_concrete.head())


print("Estadísticas descriptivas:")
print(df_concrete.describe().transpose())

# Comprobar valores nulos 
print("Valores nulos por columna:")
print(df_concrete.isnull().sum())

Preparamos los datos para poder generar los modelos

In [ ]:
#1. Separamos las entradas de las salidas
df_concrete_inputs = df_concrete.drop('concrete_compressive_strength', axis=1)
df_concrete_outputs = df_concrete['concrete_compressive_strength']


# 2. Dividimos en entrenamiento, validación y test (test)
# Usaremos un 70% para entrenamiento y  un 15% para Validación y Test.
# random_state asegura reproducibilidad.
SEED=42
X_train, X_pruebas, y_train, y_pruebas = train_test_split(df_concrete_inputs, df_concrete_outputs, test_size=0.30, random_state=SEED)


X_val, X_test, y_val, y_test = train_test_split(X_pruebas, y_pruebas, test_size=0.50, random_state=SEED)



print("Datos de entrenamiento:", X_train.shape, y_train.shape)
print("Datos de validación:", X_val.shape, y_val.shape)
print("Datos de test:", X_test.shape, y_test.shape)

# 3. Escalado de las características 
# Usaremos StandardScaler: Z = (x - mean) / std_dev
# Es crucial ajustar el escalador SÓLO con los datos de entrenamiento

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val) # Usar el mismo scaler ajustado
X_test_scaled = scaler.transform(X_test) # Usar el mismo scaler ajustado

# Convertimos a DataFrames para mejor visualización (opcional)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=df_concrete_inputs.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=df_concrete_inputs.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=df_concrete_inputs.columns)

print("Primeras filas de X_train escalado:")
print(X_train_scaled.head())
print("Media de X_train escalado (debería ser cercana a 0):")
print(X_train_scaled.mean())
print("Desviación estándar de X_train escalado (debería ser cercana a 1):")
print(X_train_scaled.std())



## Construcción del modelo

Keras permite general modelos empleando dos opciones:
1. El [API secuencial](https://keras.io/guides/sequential_model/): Permite generar redes densas con diferentes capas donde cada capa tiene exactamente un tensor de entrada y un tensor de salida.
2. El [API funcional](https://keras.io/guides/functional_api/): permite crear modelos más flexibles que los secuenciales. Esta API permite gestionar modelos con topologías no lineales. Permite crear "grafos de capas".



Vamos a definir nuestra red neuronal usando el API Secuencial de Keras (`keras.Sequential`), que es suficiente crear un modelo que "apile"  capas una tras otra.


Definiremos una red sencilla:
- Capa de Entrada: Definida implícitamente por input_shape en la primera capa densa. Debe coincidir con el número de características (8 en nuestro caso).
- Capas Ocultas: Usaremos un par de capas densas (`keras.layers.Dense`) con activación ReLU (`relu`). ReLU es una opción común y eficiente para capas ocultas. El número de neuronas es un hiperparámetro a optimizar (empezaremos con 64 y 32).
- Capa de Salida: Una única neurona (Dense(1)) ya que es un problema de regresión (predecir un solo valor continuo). No usaremos función de activación (o equivalentemente, activación 'linear'), porque queremos que la salida pueda tomar cualquier valor real, no limitado a un rango específico.

Explicación del código:
- keras.Sequential([...]): Crea un modelo donde las capas se ejecutan en secuencia.
- keras.Input(shape=(n_features,)): Define formalmente la forma de los datos de entrada. Ayuda a Keras a construir el modelo correctamente.
- keras.layers.Dense(units, activation=...): Define una capa totalmente conectada (densa). units es el número de neuronas. activation es la función de activación.
- model.summary(): Muestra las capas, la forma de salida de cada capa y el número de parámetros entrenables. Es útil para verificar la arquitectura.
    
   

In [ ]:
 # Obtenemos el número de características para la capa de entrada
n_features = X_train_scaled.shape[1]
print(f"Número de características de entrada: {n_features}")

# Construcción del modelo secuencial
model = keras.Sequential(
    [
        keras.Input(shape=(n_features,), name="input_layer"), # Capa de entrada explícita. No es obligatoria pero nos permite ver el resumen completo de la red
        keras.layers.Dense(64, activation='relu', name='hidden_layer_1'), # 1ra capa oculta con 64 neuronas y activación ReLU
        keras.layers.Dense(32, activation='relu', name='hidden_layer_2'), # 2da capa oculta con 32 neuronas y activación ReLU
        keras.layers.Dense(1, activation='linear', name='output_layer')   # Capa de salida con 1 neurona (regresión) y activación lineal
    ],
    name="Concrete_Strength_Predictor" # Nombre opcional para el modelo
)


# Mostrar un resumen de la arquitectura del modelo
print("\nResumen del modelo:")
model.summary()

## Compilación del modelo


Antes de entrenar, necesitamos "compilar" el modelo. Esto implica configurar:

* Optimizador: Algoritmo que ajusta los pesos de la red durante el entrenamiento(ej. Adam, SGD, RMSprop). Adam es una opción robusta y popular. Tenéis [aquí](https://keras.io/api/optimizers/) un listado de los disponibles.

* Función de Pérdida (*Loss Function*): Mide qué tan bien se desempeña el modelo durante el entrenamiento. Para regresión, mean_squared_error (MSE) es una opción muy típica. Tenéis [aquí](https://keras.io/api/losses/) un listado de los disponibles.


* Métricas: Funciones para evaluar el rendimiento del modelo (ej. mean_absolute_error - MAE). **Las métricas no se usan para optimizar el modelo, solo para monitorizarlo**. Tenéis [aquí](https://keras.io/api/metrics/) un listado de las disponibles

In [ ]:
model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001), # Optimizador Adam con tasa de aprendizaje típica
        loss='mean_squared_error',          # Función de pérdida para regresión (MSE)
        metrics=[keras.metrics.MeanAbsoluteError(name='mae')] # Métrica para monitorear (MAE)
    )

print("Modelo compilado exitosamente.")
print(f"Optimizador: {type(model.optimizer).__name__}")
print(f"Función de pérdida: {model.loss}")
print("Métricas: ")
#metricas incluye la empleada como función de pérdida
for m in model.metrics:
    print(m.name)


## Entrenamiento del modelo
Ahora entrenamos el modelo con nuestros datos de entrenamiento escalados. 
El método `fit` nos permite hacerlo. Tenéis [aquí](https://keras.io/api/models/model_training_apis/#fit-method) disponible todos los parámetros posibles. en este caso emplearemos:

* epochs: Número de veces que el modelo verá el conjunto de datos completo.
* batch_size: Número de muestras procesadas antes de actualizar los pesos (entrenamiento con *mini-batches*).
* validation_data: conjunto de datos sobre los que validaremos el entrenamiento del modelo. Lo podemos emplear para comparar configuraciones o para parar el entrenamiento de forma prematura (*early_stopping*), lo que puede ayudar a detectar el *overfitting*.

El método `fit` devuelve un objeto *History* que contiene información sobre el proceso de entrenamiento (pérdida y métricas en cada época).

In [ ]:

# Definimos el número de épocas y el tamaño del batch
EPOCHS = 100
BATCH_SIZE = 32

# Entrenamos el modelo
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1 # Muestra una barra de progreso por época (0=silencioso, 1=barra, 2=una línea por época)
)

## Evaluación del Modelo

Evaluaremos el rendimiento del modelo de dos maneras:

1. Visualizando el historial de entrenamiento: Graficaremos la pérdida y la métrica (MAE) tanto para el conjunto de entrenamiento como para el de validación a lo largo de las épocas. Esto nos ayuda a diagnosticar problemas como el overfitting (cuando la pérdida de validación empieza a aumentar mientras la de entrenamiento sigue bajando) o el underfitting (cuando ambas pérdidas son altas).
2. Evaluando en el conjunto de Test: Usaremos el método `evaluate` con los datos de Test (X_test_scaled, y_test), que el modelo nunca ha visto. Esto nos da una estimación final e imparcial del rendimiento del modelo.


In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(hist['epoch'], hist['loss'], label='Pérdida Entrenamiento')
plt.plot(hist['epoch'], hist['val_loss'], label = 'Pérdida Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.title('Pérdida (MSE) durante Entrenamiento')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
# Asegúrate de usar el nombre correcto de la métrica (puede variar ligeramente)
mae_key = 'mae' if 'mae' in hist.columns else list(hist.columns)[1] # Intenta encontrar la clave MAE
val_mae_key = 'val_' + mae_key
plt.plot(hist['epoch'], hist[mae_key], label='MAE Entrenamiento')
plt.plot(hist['epoch'], hist[val_mae_key], label = 'MAE Validación')
plt.xlabel('Épocas')
plt.ylabel('Error Absoluto Medio (MAE)')
plt.title('MAE durante Entrenamiento')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 2. Evaluar en el conjunto de prueba
print("\nEvaluando el modelo en el conjunto de prueba...")
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"Pérdida en el conjunto de Test (MSE): {test_loss:.4f}")
print(f"Error Absoluto Medio (MAE) en el conjunto de Test: {test_mae:.4f}")

# Calcular R^2 score (opcional, pero útil en regresión)
y_pred_test = model.predict(X_test_scaled).flatten() # Aplanar para que tenga la misma forma que y_test
r2 = r2_score(y_test, y_pred_test)
print(f"Coeficiente de Determinación (R²) en el conjunto de Test: {r2:.4f}")
